# MetaPop: Metagenomic Population Diversity Analysis

This notebook provides an interactive interface for running MetaPop on Google Colab.

**MetaPop** analyzes:
- **Macrodiversity**: Sample-level diversity (richness, Shannon's H, Simpson, etc.)
- **Microdiversity**: Within-population genetic variation (pi, theta, Tajima's D, pN/pS)

---

## 1. Setup and Installation

Run this cell first to install all required dependencies.

In [ ]:
# Step 1: Install system dependencies (samtools, bcftools, prodigal)
!apt-get update -qq 2>/dev/null
!apt-get install -y -qq samtools bcftools prodigal 2>/dev/null

# Step 2: Install Python dependencies
!pip install -q numpy pandas scipy pysam matplotlib seaborn scikit-learn plotly ipywidgets

# Step 3: Clone and install MetaPop
import os
if not os.path.exists('metapop'):
    !git clone https://github.com/espickle1/metapop.git

%cd metapop
!pip install -q .
%cd ..

print("Base dependencies and MetaPop installed!")

## 2. Mount Google Drive (Optional)

Mount your Google Drive to access data files and save results.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Your files will be available at /content/drive/MyDrive/

In [ ]:
import shutil
import os

# Define paths
drive_bam_dir = '/content/drive/MyDrive/metapop_data/bams'  # Your Drive path
local_bam_dir = '/content/local_bams'  # Local Colab storage

# Create local directory
os.makedirs(local_bam_dir, exist_ok=True)

# Copy all BAM files to local storage
print("Copying BAM files to local storage (this may take a while)...")
for f in os.listdir(drive_bam_dir):
    if f.endswith('.bam') or f.endswith('.bai'):
        src = os.path.join(drive_bam_dir, f)
        dst = os.path.join(local_bam_dir, f)
        if not os.path.exists(dst):
            print(f"  Copying {f}...")
            shutil.copy2(src, dst)
        else:
            print(f"  {f} already exists locally")

print(f"\nDone! Use '{local_bam_dir}' as your BAM Directory in the widget above.")

# Verify with samtools
!samtools view -c {local_bam_dir}/*.bam | head -1

## 3. Import Libraries and Initialize

In [ ]:
import os
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output

# Try to import plotly for interactive plots
try:
    import plotly.express as px
    import plotly.graph_objects as go
    PLOTLY_AVAILABLE = True
except ImportError:
    PLOTLY_AVAILABLE = False

print("Libraries loaded successfully!")
print(f"Interactive plots (Plotly): {'Available' if PLOTLY_AVAILABLE else 'Not available'}")

## 4. Configure Pipeline Parameters

Use the interactive widgets below to configure your analysis parameters.

In [ ]:
# File path inputs with live validation and dark text
import ipywidgets as widgets
from IPython.display import display, HTML

# Add CSS to make widget text darker and more readable
display(HTML('''
<style>
.jupyter-widgets .widget-text input {
    color: #000000 !important;
    font-weight: 500;
}
.jupyter-widgets .widget-label {
    color: #000000 !important;
    font-weight: bold;
}
.jupyter-widgets .widget-inline-hbox .widget-label,
.jupyter-widgets .widget-inline-vbox .widget-label {
    color: #000000 !important;
    font-weight: bold;
}
</style>
'''))

print("=== File Paths ===")
print("Configure your input/output directories below.")
print()

input_dir = widgets.Text(
    value='/content/drive/MyDrive/metapop_data/bams',
    description='BAM Directory:',
    style={'description_width': '150px', 'description_color': '#000000'},
    layout=widgets.Layout(width='90%'),
    continuous_update=True
)

reference = widgets.Text(
    value='/content/drive/MyDrive/metapop_data/references',
    description='Reference Directory:',
    placeholder='Directory containing reference FASTA files',
    style={'description_width': '150px', 'description_color': '#000000'},
    layout=widgets.Layout(width='90%'),
    continuous_update=True
)

genes = widgets.Text(
    value='',
    description='Genes File (optional):',
    placeholder='Leave empty to auto-generate with Prodigal',
    style={'description_width': '150px', 'description_color': '#000000'},
    layout=widgets.Layout(width='90%'),
    continuous_update=True
)

norm_file = widgets.Text(
    value='',
    description='Normalization File:',
    placeholder='Leave empty to auto-generate from BAM read counts',
    style={'description_width': '150px', 'description_color': '#000000'},
    layout=widgets.Layout(width='90%'),
    continuous_update=True
)

output_dir = widgets.Text(
    value='/content/drive/MyDrive/metapop_results',
    description='Output Directory:',
    style={'description_width': '150px', 'description_color': '#000000'},
    layout=widgets.Layout(width='90%'),
    continuous_update=True
)

# Status indicator for path validation
path_status = widgets.HTML(value='')

def validate_paths(change=None):
    """Validate paths and update status."""
    status_parts = []
    
    # Check BAM directory
    if os.path.exists(input_dir.value):
        bam_files = [f for f in os.listdir(input_dir.value) if f.endswith('.bam')]
        status_parts.append(f"✅ BAM directory: {len(bam_files)} BAM file(s) found")
    else:
        status_parts.append(f"❌ BAM directory not found")
    
    # Check reference directory
    if os.path.exists(reference.value):
        if os.path.isdir(reference.value):
            ref_files = [f for f in os.listdir(reference.value) 
                        if f.endswith(('.fa', '.fasta', '.fna'))]
            status_parts.append(f"✅ Reference directory: {len(ref_files)} FASTA file(s) found")
        else:
            status_parts.append(f"⚠️ Reference is a file, not directory")
    else:
        status_parts.append(f"❌ Reference directory not found")
    
    # Check output directory
    if os.path.exists(output_dir.value):
        status_parts.append(f"✅ Output directory exists")
    else:
        status_parts.append(f"ℹ️ Output directory will be created")
    
    path_status.value = '<br>'.join(status_parts)

# Connect validation to path changes
input_dir.observe(validate_paths, names='value')
reference.observe(validate_paths, names='value')
output_dir.observe(validate_paths, names='value')

# Initial validation
validate_paths()

# Display widgets
display(widgets.VBox([
    input_dir,
    reference, 
    genes,
    norm_file,
    output_dir,
    widgets.HTML('<hr style="margin: 10px 0;">'),
    widgets.HTML('<b>Path Status:</b>'),
    path_status
]))

In [ ]:
# Filter parameters with live value display and dark text
print("=== Filter Parameters ===")
print("Adjust the sliders below. Values update in real-time.")
print()

min_pct_id = widgets.FloatSlider(
    value=95.0, min=80.0, max=100.0, step=0.5,
    description='Min % Identity:',
    style={'description_width': '150px', 'description_color': '#000000'},
    layout=widgets.Layout(width='70%'),
    readout=True,
    readout_format='.1f',
    continuous_update=True
)

min_length = widgets.IntSlider(
    value=50, min=25, max=200, step=5,
    description='Min Read Length:',
    style={'description_width': '150px', 'description_color': '#000000'},
    layout=widgets.Layout(width='70%'),
    readout=True,
    continuous_update=True
)

min_cov = widgets.IntSlider(
    value=20, min=0, max=100, step=5,
    description='Min Coverage (%):',
    style={'description_width': '150px', 'description_color': '#000000'},
    layout=widgets.Layout(width='70%'),
    readout=True,
    continuous_update=True
)

min_dep = widgets.IntSlider(
    value=10, min=1, max=100, step=1,
    description='Min Depth:',
    style={'description_width': '150px', 'description_color': '#000000'},
    layout=widgets.Layout(width='70%'),
    readout=True,
    continuous_update=True
)

truncation = widgets.FloatSlider(
    value=10.0, min=0.0, max=49.0, step=1.0,
    description='Truncation (%):',
    style={'description_width': '150px', 'description_color': '#000000'},
    layout=widgets.Layout(width='70%'),
    readout=True,
    readout_format='.1f',
    continuous_update=True
)

# Live summary of filter settings
filter_summary = widgets.HTML(value='')

def update_filter_summary(change=None):
    """Update the filter summary display."""
    summary = f"""
    <div style="background-color: #f0f0f0; padding: 10px; border-radius: 5px; margin-top: 10px; color: #000000;">
    <b>Current Filter Settings:</b><br>
    • Reads must have ≥{min_pct_id.value:.1f}% identity to reference<br>
    • Reads must be ≥{min_length.value} bp long<br>
    • Contigs need ≥{min_cov.value}% coverage breadth<br>
    • Positions need ≥{min_dep.value}x read depth<br>
    • Truncate {truncation.value:.0f}% from each contig end
    </div>
    """
    filter_summary.value = summary

# Connect summary update to all sliders
for slider in [min_pct_id, min_length, min_cov, min_dep, truncation]:
    slider.observe(update_filter_summary, names='value')

# Initial summary
update_filter_summary()

# Display with organized layout
display(widgets.VBox([
    min_pct_id,
    min_length,
    min_cov,
    min_dep,
    truncation,
    filter_summary
]))

In [ ]:
# Analysis options with live summary and dark text
print("=== Analysis Options ===")
print("Select which analyses to run and configure threads.")
print()

run_preproc = widgets.Checkbox(
    value=True, 
    description='Run Preprocessing',
    style={'description_width': 'initial', 'description_color': '#000000'},
    indent=False
)

run_microdiv = widgets.Checkbox(
    value=True, 
    description='Run Microdiversity Analysis',
    style={'description_width': 'initial', 'description_color': '#000000'},
    indent=False
)

run_macrodiv = widgets.Checkbox(
    value=True, 
    description='Run Macrodiversity Analysis',
    style={'description_width': 'initial', 'description_color': '#000000'},
    indent=False
)

run_viz = widgets.Checkbox(
    value=True, 
    description='Generate Visualizations',
    style={'description_width': 'initial', 'description_color': '#000000'},
    indent=False
)

threads = widgets.IntSlider(
    value=4, min=1, max=16, step=1,
    description='Threads:',
    style={'description_width': '150px', 'description_color': '#000000'},
    layout=widgets.Layout(width='50%'),
    readout=True,
    continuous_update=True
)

# Live summary of what will run
analysis_summary = widgets.HTML(value='')

def update_analysis_summary(change=None):
    """Update the analysis summary."""
    analyses = []
    if run_preproc.value:
        analyses.append("✅ Preprocessing (filter reads, compute coverage)")
    else:
        analyses.append("⏭️ Skip preprocessing")
    
    if run_microdiv.value:
        analyses.append("✅ Microdiversity (SNPs, pi, theta, Tajima's D, pN/pS)")
    else:
        analyses.append("⏭️ Skip microdiversity")
    
    if run_macrodiv.value:
        analyses.append("✅ Macrodiversity (richness, Shannon's H, beta diversity)")
    else:
        analyses.append("⏭️ Skip macrodiversity")
    
    if run_viz.value:
        analyses.append("✅ Visualizations (plots and figures)")
    else:
        analyses.append("⏭️ Skip visualizations")
    
    summary = f"""
    <div style="background-color: #e8f4e8; padding: 10px; border-radius: 5px; margin-top: 10px; color: #000000;">
    <b>Pipeline Configuration:</b><br>
    {'<br>'.join(analyses)}<br>
    <br>
    <b>Using {threads.value} thread(s)</b> for parallel processing
    </div>
    """
    analysis_summary.value = summary

# Connect to all checkboxes and threads
for widget in [run_preproc, run_microdiv, run_macrodiv, run_viz, threads]:
    widget.observe(update_analysis_summary, names='value')

# Initial summary
update_analysis_summary()

# Organize checkboxes in columns
checkbox_box = widgets.HBox([
    widgets.VBox([run_preproc, run_microdiv]),
    widgets.VBox([run_macrodiv, run_viz])
])

display(widgets.VBox([
    checkbox_box,
    widgets.HTML('<br>'),
    threads,
    analysis_summary
]))

## 5. Run MetaPop Pipeline

In [ ]:
import subprocess

def run_metapop():
    """Run the MetaPop pipeline with configured parameters."""
    clear_output()
    print("Starting MetaPop Pipeline...")
    print("="*50)
    
    # Validate inputs
    if not os.path.exists(input_dir.value):
        print(f"Error: BAM directory not found: {input_dir.value}")
        return
    
    # Handle reference path - MetaPop expects a directory
    ref_path = reference.value
    if os.path.isfile(ref_path):
        print(f"Warning: Reference path is a file, not a directory.")
        print(f"File: {ref_path}")
        print(f"MetaPop expects a directory containing FASTA files.")
        print(f"Using parent directory: {os.path.dirname(ref_path)}")
        ref_path = os.path.dirname(ref_path)
        print()
    
    if not os.path.exists(ref_path):
        print(f"Error: Reference directory not found: {ref_path}")
        return
    
    if not os.path.isdir(ref_path):
        print(f"Error: Reference path is not a directory: {ref_path}")
        return
    
    # Create output directory
    os.makedirs(output_dir.value, exist_ok=True)
    
    # Build command
    cmd = [
        'metapop',
        '--input_samples', input_dir.value,
        '--reference', ref_path,
        '--output', output_dir.value,
        '--id_min', str(min_pct_id.value),
        '--min_len', str(min_length.value),
        '--min_cov', str(min_cov.value),
        '--min_dep', str(min_dep.value),
        '--trunc', str(truncation.value),
        '--threads', str(threads.value),
    ]
    
    if genes.value:
        cmd.extend(['--genes', genes.value])
    if norm_file.value:
        cmd.extend(['--norm', norm_file.value])
    if not run_microdiv.value:
        cmd.append('--no_micro')
    if not run_macrodiv.value:
        cmd.append('--no_macro')
    if not run_viz.value:
        cmd.append('--no_viz')
    
    print(f"Configuration:")
    print(f"  Input: {input_dir.value}")
    print(f"  Reference: {ref_path}")
    print(f"  Output: {output_dir.value}")
    print(f"  Threads: {threads.value}")
    print("="*50)
    print("\nRunning MetaPop...")
    print()
    
    # Run the pipeline
    try:
        process = subprocess.Popen(
            cmd,
            stdout=subprocess.PIPE,
            stderr=subprocess.STDOUT,
            text=True,
            bufsize=1
        )
        
        # Stream output in real-time
        for line in process.stdout:
            print(line, end='')
        
        process.wait()
        
        if process.returncode == 0:
            print("\n" + "="*50)
            print("MetaPop pipeline completed successfully!")
            print(f"Results saved to: {output_dir.value}")
        else:
            print("\n" + "="*50)
            print(f"MetaPop exited with error code: {process.returncode}")
            
    except FileNotFoundError:
        print("\nError: MetaPop command not found.")
        print("Please make sure MetaPop is installed correctly.")
        print("Try running: !pip install -e /content/metapop")
    except Exception as e:
        print(f"\nError during pipeline execution: {e}")
        import traceback
        traceback.print_exc()

# Create run button
run_button = widgets.Button(
    description='Run MetaPop',
    button_style='success',
    tooltip='Click to start the pipeline',
    icon='play'
)
run_button.on_click(lambda b: run_metapop())

display(run_button)

## 6. View Results

After the pipeline completes, use the cells below to explore results interactively.

In [ ]:
def load_results():
    """Load and display MetaPop results."""
    results_path = os.path.join(output_dir.value, 'MetaPop')
    
    if not os.path.exists(results_path):
        print(f"Results not found at: {results_path}")
        print("Please run the pipeline first.")
        return None
    
    results = {}
    
    # Load alpha diversity
    alpha_file = os.path.join(results_path, '11.Macrodiversity', 'Alpha_diversity_stats.tsv')
    if os.path.exists(alpha_file):
        results['alpha'] = pd.read_csv(alpha_file, sep='\t', index_col=0)
        print(f"Loaded alpha diversity: {len(results['alpha'])} samples")
    
    # Load gene microdiversity
    gene_micro_file = os.path.join(results_path, '10.Microdiversity', 'global_gene_microdiversity.tsv')
    if os.path.exists(gene_micro_file):
        results['gene_micro'] = pd.read_csv(gene_micro_file, sep='\t')
        print(f"Loaded gene microdiversity: {len(results['gene_micro'])} genes")
    
    # Load abundance matrix
    abundance_file = os.path.join(results_path, '11.Macrodiversity', 'normalized_abundances_table.tsv')
    if os.path.exists(abundance_file):
        results['abundance'] = pd.read_csv(abundance_file, sep='\t', index_col=0)
        print(f"Loaded abundance matrix: {results['abundance'].shape}")
    
    return results

results = load_results()

In [ ]:
# Display alpha diversity summary
if results and 'alpha' in results:
    print("Alpha Diversity Summary:")
    display(results['alpha'].describe())
    
    if PLOTLY_AVAILABLE:
        # Interactive bar chart
        fig = px.bar(results['alpha'].reset_index(), x='index', y='Richness',
                    title='Species Richness by Sample',
                    labels={'index': 'Sample', 'Richness': 'Species Richness'})
        fig.show()
    else:
        # Static matplotlib plot
        plt.figure(figsize=(10, 5))
        results['alpha']['Richness'].plot(kind='bar')
        plt.title('Species Richness by Sample')
        plt.ylabel('Richness')
        plt.tight_layout()
        plt.show()

In [ ]:
# Display pN/pS distribution
if results and 'gene_micro' in results:
    pnps = results['gene_micro']['pNpS_ratio'].dropna()
    pnps = pnps[~np.isinf(pnps) & (pnps < 10)]  # Filter outliers
    
    print(f"pN/pS Statistics:")
    print(f"  Mean: {pnps.mean():.3f}")
    print(f"  Median: {pnps.median():.3f}")
    print(f"  Genes with pN/pS < 1 (purifying): {(pnps < 1).sum()}")
    print(f"  Genes with pN/pS > 1 (positive): {(pnps > 1).sum()}")
    
    if PLOTLY_AVAILABLE:
        fig = px.histogram(pnps, nbins=50, title='pN/pS Ratio Distribution',
                          labels={'value': 'pN/pS Ratio', 'count': 'Gene Count'})
        fig.add_vline(x=1, line_dash='dash', line_color='red',
                     annotation_text='Neutral (pN/pS=1)')
        fig.show()
    else:
        plt.figure(figsize=(10, 5))
        plt.hist(pnps, bins=50, edgecolor='black')
        plt.axvline(1, color='red', linestyle='--', label='Neutral')
        plt.xlabel('pN/pS Ratio')
        plt.ylabel('Gene Count')
        plt.title('pN/pS Distribution')
        plt.legend()
        plt.tight_layout()
        plt.show()

## 7. Download Results

Download the results as a ZIP file.

In [ ]:
from google.colab import files
import shutil

def download_results():
    """Create ZIP archive and download results."""
    results_path = os.path.join(output_dir.value, 'MetaPop')
    
    if not os.path.exists(results_path):
        print("No results to download.")
        return
    
    zip_path = '/content/metapop_results'
    shutil.make_archive(zip_path, 'zip', results_path)
    
    print(f"Downloading {zip_path}.zip...")
    files.download(f'{zip_path}.zip')

download_button = widgets.Button(
    description='Download Results',
    button_style='info',
    tooltip='Download results as ZIP',
    icon='download'
)
download_button.on_click(lambda b: download_results())

display(download_button)

---

## Need Help?

- **Documentation**: See the MetaPop README for detailed usage instructions
- **Issues**: Report bugs at https://github.com/espickle1/metapop/issues
- **Contact**: James.Chang@bcm.edu